<a id="description"></a>
# <div style="text-align:center; border-radius:15px 50px; padding:15px; color:white; margin:0; font-size:100%; font-family:Pacifico; background-color:#d5f5e3; overflow:hidden"><b>Project Description</b></div>


<div style="padding: 20px; border-color: #50A20E; border-radius: 10px; box-shadow: 0 2px 4px 0 rgba(0, 0, 0, 0.1); border: 2px solid #50A20E;">
    <p>
        Machine Translation using an LSTM-based Encoder-Decoder model to translate English to Finnish!  This character-level recurrent sequence-to-sequence model was a deep dive into NLP and sequence modeling.



</p>
</div>

<a id="1"></a>
# <div style="text-align:center; border-radius:15px 50px; padding:15px; color:white; margin:0; font-size:100%; font-family:Pacifico; background-color:#d5f5e3; overflow:hidden"><b> Importing Libraries</b></div>

In [ ]:
import numpy as np

import keras
import os
from pathlib import Path

<a id="1"></a>
# <div style="text-align:center; border-radius:15px 50px; padding:15px; color:white; margin:0; font-size:100%; font-family:Pacifico; background-color:#d5f5e3; overflow:hidden"><b> Reading Data</b></div>

In [ ]:
batch_size=32
epochs=100
latent_dim=256
num_samples=12522
data_path='/content/drive/MyDrive/fra.txt'


In [ ]:
with open (data_path,"r",encoding='utf-8') as f:
     lines=f.read().split('\n')   # Read and split file into lines
     print(len(lines))            # Print total number of lines
     for i in range(10):          # Print the first 10 lines
      print(lines[i])

232737
Go.	Va !	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1158250 (Wittydev)
Go.	Marche.	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8090732 (Micsmithel)
Go.	En route !	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8267435 (felix63)
Go.	Bouge !	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #9022935 (Micsmithel)
Hi.	Salut !	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #509819 (Aiji)
Hi.	Salut.	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #4320462 (gillux)
Run!	Cours !	CC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #906331 (sacredceltic)
Run!	Courez !	CC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #906332 (sacredceltic)
Run!	Prenez vos jambes à vos cous !	CC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #2077449 (sacredceltic)
Run!	File !	CC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #2077454 (sacredceltic)


<a id="1"></a>
# <div style="text-align:center; border-radius:15px 50px; padding:15px; color:white; margin:0; font-size:100%; font-family:Pacifico; background-color:#d5f5e3; overflow:hidden"><b> Data Preprocessing</b></div>

In [ ]:
input_texts = []  # Store input sentences
target_texts = [] # Store target sentences
input_characters = set()  # Store unique input chars
target_characters = set() # Store unique target chars

with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")  # Read and split lines

for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")  # Split line into input, target
    target_text = "\t" + target_text + "\n"  # Add start/end sequence
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)  # Add unique input char
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char) # Add unique target char

input_characters = sorted(list(input_characters)) # Sort & list input chars
target_characters = sorted(list(target_characters)) # Sort & list target chars
num_encoder_tokens = len(input_characters)  # Count input chars
num_decoder_tokens = len(target_characters) # Count target chars
max_encoder_seq_length = max([len(txt) for txt in input_texts]) # Max input len
max_decoder_seq_length = max([len(txt) for txt in target_texts]) # Max target len

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 12522
Number of unique input tokens: 71
Number of unique output tokens: 93
Max sequence length for inputs: 15
Max sequence length for outputs: 59


In [ ]:
target_text

'\tMon père est occupé.\n'

In [ ]:
input_characters

[' ',
 '!',
 '"',
 '$',
 '%',
 '&',
 "'",
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '5',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 'é']

In [ ]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)]) # Create input char-to-index mapping
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)]) # Create target char-to-index mapping

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32'
)  # Initialize encoder input data
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32'
)  # Initialize decoder input data
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32'
)  # Initialize decoder target data

In [ ]:
encoder_input_data.shape,decoder_input_data.shape,decoder_target_data.shape

((12522, 15, 71), (12522, 59, 93), (12522, 59, 93))

In [ ]:
input_token_index

{' ': 0,
 '!': 1,
 '"': 2,
 '$': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '1': 11,
 '2': 12,
 '3': 13,
 '5': 14,
 '7': 15,
 '8': 16,
 '9': 17,
 ':': 18,
 '?': 19,
 'A': 20,
 'B': 21,
 'C': 22,
 'D': 23,
 'E': 24,
 'F': 25,
 'G': 26,
 'H': 27,
 'I': 28,
 'J': 29,
 'K': 30,
 'L': 31,
 'M': 32,
 'N': 33,
 'O': 34,
 'P': 35,
 'Q': 36,
 'R': 37,
 'S': 38,
 'T': 39,
 'U': 40,
 'V': 41,
 'W': 42,
 'Y': 43,
 'a': 44,
 'b': 45,
 'c': 46,
 'd': 47,
 'e': 48,
 'f': 49,
 'g': 50,
 'h': 51,
 'i': 52,
 'j': 53,
 'k': 54,
 'l': 55,
 'm': 56,
 'n': 57,
 'o': 58,
 'p': 59,
 'q': 60,
 'r': 61,
 's': 62,
 't': 63,
 'u': 64,
 'v': 65,
 'w': 66,
 'x': 67,
 'y': 68,
 'z': 69,
 'é': 70}

In [ ]:
input_token_index['R']

37

In [ ]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
  for t, char in enumerate(input_text):
    encoder_input_data[i, t, input_token_index[char]] = 1  # One-hot encode input
  encoder_input_data[i, t + 1:, input_token_index[" "]] = 1  # Pad with spaces
  for t, char in enumerate(target_text):
    decoder_input_data[i, t, target_token_index[char]] = 1  # One-hot encode target
    if t > 0:
      decoder_target_data[i, t - 1, target_token_index[char]] = 1  # Shift target for teacher forcing
  decoder_input_data[i, t + 1:, target_token_index[" "]] = 1.0  # Pad with spaces
  decoder_target_data[i, t:, target_token_index[" "]] = 1.0 # Pad with spaces

<a id="1"></a>
# <div style="text-align:center; border-radius:15px 50px; padding:15px; color:white; margin:0; font-size:100%; font-family:Pacifico; background-color:#d5f5e3; overflow:hidden"><b> Building The Model</b></div>

In [ ]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

<a id="1"></a>
# <div style="text-align:center; border-radius:15px 50px; padding:15px; color:white; margin:0; font-size:100%; font-family:Pacifico; background-color:#d5f5e3; overflow:hidden"><b> Learning Rate Scheduler & Early Stopping </b></div>

In [ ]:
from keras.callbacks import ReduceLROnPlateau

lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    verbose=1,
    min_lr=1e-6
)  # Define learning rate scheduler

from keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)  # Define early stopping callback


model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)  # Compile the model

<a id="1"></a>
# <div style="text-align:center; border-radius:15px 50px; padding:15px; color:white; margin:0; font-size:100%; font-family:Pacifico; background-color:#d5f5e3; overflow:hidden"><b> Model Training</b></div>

In [ ]:


model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
    callbacks=[lr_scheduler, early_stop]
)  # Train the model

model.save('S2S_model.keras')  # Save the trained model

Epoch 1/100
314/314 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.7183 - loss: 1.4721 - val_accuracy: 0.7540 - val_loss: 0.8979 - learning_rate: 0.0010
Epoch 2/100
314/314 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.8046 - loss: 0.7117 - val_accuracy: 0.7960 - val_loss: 0.6988 - learning_rate: 0.0010
Epoch 3/100
314/314 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.8346 - loss: 0.5710 - val_accuracy: 0.8138 - val_loss: 0.6346 - learning_rate: 0.0010
Epoch 4/100
314/314 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.8485 - loss: 0.5175 - val_accuracy: 0.8237 - val_loss: 0.6039 - learning_rate: 0.0010
Epoch 5/100
314/314 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.8562 - loss: 0.4886 - val_accuracy: 0.8322 - val_loss: 0.5699 - learning_rate: 0.0010
Epoch 6/100
314/314 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.8645 - loss: 0.4578 - val_accuracy: 0.8295 - val_loss: 0.5749 - learning_rate: 0.0010
Epoch 7/100
314/314 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.8675 - lo

<a id="1"></a>
# <div style="text-align:center; border-radius:15px 50px; padding:15px; color:white; margin:0; font-size:100%; font-family:Pacifico; background-color:#d5f5e3; overflow:hidden"><b> inference (sampling)</b></div>

In [ ]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("S2S_model.keras")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq, verbose=0)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value, verbose=0
        )

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

<a id="1"></a>
# <div style="text-align:center; border-radius:15px 50px; padding:15px; color:white; margin:0; font-size:100%; font-family:Pacifico; background-color:#d5f5e3; overflow:hidden"><b> Generate Decoded Sentences</b></div>

In [ ]:
for seq_index in range(50):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

-
Input sentence: Go.
Decoded sentence: Cours !

-
Input sentence: Go.
Decoded sentence: Cours !

-
Input sentence: Go.
Decoded sentence: Cours !

-
Input sentence: Go.
Decoded sentence: Cours !

-
Input sentence: Hi.
Decoded sentence: Salut !

-
Input sentence: Hi.
Decoded sentence: Salut !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run.
Decoded sentence: Fuyez !

-
Input sentence: Run.
Decoded sentence: Fuyez !

-
Input sentence: Run.
Decoded sentence: Fuyez !

-
Input sentence: Run.
Decoded sentence: Fuyez !

-
Input sentence: Run.
Decoded sentence: Fuyez !

-
Input sentence: Run.
Decoded sentence: Fuyez !

-
Inpu